In [ ]:
import pandas as pd
import os
import sqlite3 as sql
import numpy as np

from functions import fetch_data

In [ ]:
import warnings
warnings.simplefilter('ignore')

Data for the backtesting environment will come from Yahoo Finance:

In [ ]:
#path = 'C:\\Users\\William\\Downloads'
path = '\\Downloads'
print('https://finance.yahoo.com/lookup/')

Pull files from downloads to save as pandas dataframes:

In [ ]:
dataframes = {}
files = ['FB.csv', 'AAPL.csv', 'NFLX.csv', 'GOOG.csv', 'ES=F.csv']

for file in files:
    
    df = pd.read_csv(os.path.join(path, file))
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')

    df.sort_values('date', inplace=True)
    df.drop('adj_close', axis=1, inplace=True)

    if file == 'ES=F.csv':
        name = 'sp500_df'
    else:
        name = file[:-4].lower() + '_df'
    
    dataframes[name] = df

dataframes.keys()

Drop observations before 2009:

In [ ]:
dataframes['aapl_df'] = dataframes['aapl_df'].loc[7078:].dropna(axis=0)

In [ ]:
dataframes['nflx_df'] = dataframes['nflx_df'].loc[1664:].dropna(axis=0)

In [ ]:
dataframes['goog_df'] = dataframes['goog_df'].loc[1100:].dropna(axis=0)

In [ ]:
dataframes['sp500_df'] = dataframes['sp500_df'].loc[2577:].dropna(axis=0)

Function for building the db:

In [ ]:
def build_sql(conn, all_dfs):

    for name, df in all_dfs.items():

        df.to_sql(name, con=conn, index=False, if_exists='replace')
    
    conn.commit()

Function for querying/testing the db:

In [ ]:
def query(conn, name):
    
    c = conn.cursor()
    
    c.execute(f"""
        SELECT * 
        FROM {name};
        """)
    
    df = pd.DataFrame(c.fetchall(), columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'])
    
    print(df.head())

I like to wrap SQL functions in try-except clauses to avoid certain errors with the .db file

In [ ]:
connection = sql.connect('ticker_data.db')

try:
    build_sql(connection, dataframes)
    #query(connection, 'fb_df')
except Exception as e:
    print('EXCEPTION', e)

connection.close()

In [ ]:
connection = sql.connect('ticker_data.db')

try:
    query(connection, 'sp500_df')
except Exception as e:
    print('EXCEPTION', e)

connection.close()

In [ ]:
fetch_data('fb_df').head()